In [2]:
import BioSimSpace as bss
import csv
import pandas as pd
import glob
import networkx as nx
import matplotlib.pyplot as plt
import os
import MDAnalysis as mda
import nglview as nv

/home/jguven/Software/miniconda3/envs/sire/lib/python3.9/site-packages/numpy/core/getlimits.py:500: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/jguven/Software/miniconda3/envs/sire/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/jguven/Software/miniconda3/envs/sire/lib/python3.9/site-packages/numpy/core/getlimits.py:500: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/jguven/Software/miniconda3/envs/sire/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)



Sending anonymous Sire usage statistics to http://siremol.org.
For more information, see http://siremol.org/analytics
To disable, set the environment variable 'SIRE_DONT_PHONEHOME' to 1
To see the information sent, set the environment variable 
SIRE_VERBOSE_PHONEHOME equal to 1. To silence this message, set
the environment variable SIRE_SILENT_PHONEHOME to 1.



In [13]:
ligand_path = "../vim_2/inputs/ligands/"
csv_file = ligand_path + "lomap_vim_2.csv"
dataframe = pd.read_csv(csv_file, header=None, sep=":", names=["perturbation", "score"])
network_dict = dict(zip(dataframe["perturbation"], dataframe["score"]))

In [15]:
network_dict.keys()

dict_keys(["('docked_1', 'docked_2')", "('docked_1', 'docked_4')", "('docked_10', 'docked_4')", "('docked_10', 'docked_5')", "('docked_10', 'docked_8')", "('docked_11', 'docked_15')", "('docked_11', 'docked_5')", "('docked_12', 'docked_14')", "('docked_12', 'docked_15')", "('docked_13', 'docked_3')", "('docked_13', 'docked_6')", "('docked_13', 'docked_7')", "('docked_14', 'docked_15')", "('docked_15', 'docked_5')", "('docked_16', 'docked_2')", "('docked_16', 'docked_4')", "('docked_16', 'docked_9')", "('docked_2', 'docked_4')", "('docked_3', 'docked_6')", "('docked_4', 'docked_6')", "('docked_5', 'docked_9')", "('docked_6', 'docked_7')", "('docked_6', 'docked_8')"])

Remove some edges:


In [4]:
test_dict = {("docked_1", "docked_2"): 0.5,
             ("docked_3", "docked_4"): 0.6}
test_df = pd.DataFrame.from_dict(test_dict, "index")
new_test = test_df.reset_index().rename(columns={"index":"perts", 0: "score"})
new_test.drop([0], axis=0, inplace=True)
new_test

,perts,score
1,"(docked_3, docked_4)",0.6


In [8]:
new_test.last_valid_index()

1

In [64]:
network_dict.pop("('docked_10', 'docked_5')")
network_dict.pop("('docked_5', 'docked_9')")
network_dict.pop("('docked_16', 'docked_2')")
network_dict.pop("('docked_16', 'docked_4')")

0.12246

Add some new edges and change existing scores

In [65]:
network_dict["('docked_16', 'docked_10')"] = 0.1
network_dict["('docked_15', 'docked_9')"] = 0.1
network_dict["('docked_9', 'docked_10')"] = 0.1
network_dict["('docked_3', 'docked_6')"] = 0.1
network_dict["('docked_13', 'docked_3')"] = 0.1


In [12]:
test = [1, 2, 3]
try:
    assert len(test) == 2
except AssertionError:
    print("len should be 3")

len should be 3


In [88]:
with open(ligand_path+f"/adjusted_lomap_vim_2.csv", "w") as lomap_out:
    for key, value in network_dict.items():
        lomap_out.write(f"{key}: {value}\n")

Plot new network

In [82]:
ligand_files = sorted(glob.glob(f"{ligand_path}docked_*.sdf"))
ligand_names = [filepath.split("/")[-1].replace(".sdf","") for filepath in ligand_files]

In [87]:

# graph = nx.Graph()

# for ligand in ligand_names:
#     graph.add_node(ligand, label=ligand, labelloc="t")

# for edge in network_dict:
#     graph.add_edge(edge[0], edge[1], label=(network_dict[edge]))

# positions = nx.kamada_kawai_layout(graph)
# fig, ax = plt.subplots(figsize=(14, 14))
# nx.draw(graph, positions, ax, edge_color="black", width=1, linewidths=1, node_size=1500, node_color="skyblue", font_size=12, 
#         labels={node: node for node in graph.nodes()}, with_labels=True, connectionstyle="arc3, rad=0.1")

# # nx.draw_networkx_edge_labels(graph, positions, edge_labels=network_dict, font_color="purple", font_size=10)\
# edge_labels = network_dict
# plt.savefig("../plots/adjusted_network.png", dpi=1000, transparent=True)
# plt.show()



In [122]:

# path = "/home/jguven/projects/metalloenzymes/all_ligands/"

# ligand_files = sorted(glob.glob(f"{path}/ligand_*.sdf"))

# ligands = [bss.IO.readMolecules(filepath)[0] for filepath in ligand_files]
# ligand_names = [filepath.split("/")[-1].replace(".sdf","") for filepath in ligand_files]
# transformations, lomap_scores = bss.Align.generateNetwork(ligands, plot_network=True, names=ligand_names, work_dir=path)

# pert_network_dict = {}
# transformations_named = [(ligand_names[transf[0]], ligand_names[transf[1]]) for transf in transformations]
# for transf, score in zip(transformations_named, lomap_scores):
#     pert_network_dict[transf] = score